In [1]:
import sys, os
if os.path.abspath(os.pardir) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.pardir))
import CONFIG
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
import pydicom
import matplotlib.pyplot as plt
import cv2
import random
import torch.nn.functional as F
from sklearn import model_selection
from sklearn import preprocessing

In [41]:
DATA_DIR = CONFIG.CFG.DATA.BASE
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

QUANTILES = [0.2, 0.5, 0.8]
SCALE_COLUMNS = ['Weeks', 'FVC', 'Percent', 'Age'] #'Percent'
SCALE_COLUMNS = ['Weeks_Passed', 'Base_FVC', 'Base_Percent', 'Base_Age']
SEX_COLUMNS = ['Male', 'Female']
SMOKING_STATUS_COLUMNS = ['Currently smokes', 'Ex-smoker', 'Never smoked']
FV = SEX_COLUMNS + SMOKING_STATUS_COLUMNS + SCALE_COLUMNS

# number of images used to create a single 3D array of the scan
NUM_IMAGES = 8
IMG_SIZE = 256
K_FOLDS = 5
LEARNING_RATE = 4e-5
NUM_EPOCHS = 1000

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(2020)

In [5]:
kf = model_selection.KFold(K_FOLDS)
MIN_MAX_SCALER = preprocessing.MinMaxScaler()

In [6]:
train_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
test_df = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
sub_df = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))
# remove the duplicates from the train_df
train_df.drop_duplicates(keep=False, inplace=True, subset=['Patient', 'Weeks'])

In [7]:
# extract the Patient and weeks from the Patient_Week column
sub_df['Patient'] = sub_df['Patient_Week'].apply(lambda x: x.split('_')[0])
sub_df['Weeks'] = sub_df['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
sub_df.head()

,Patient_Week,FVC,Confidence,Patient,Weeks
0,ID00419637202311204720264_-12,2000,100,ID00419637202311204720264,-12
1,ID00421637202311550012437_-12,2000,100,ID00421637202311550012437,-12
2,ID00422637202311677017371_-12,2000,100,ID00422637202311677017371,-12
3,ID00423637202312137826377_-12,2000,100,ID00423637202312137826377,-12
4,ID00426637202313170790466_-12,2000,100,ID00426637202313170790466,-12


In [8]:
# merge the sub_df with the test_df
sub_df = sub_df.drop('FVC', axis=1).merge(test_df.drop('Weeks', axis=1), on='Patient')
sub_df.head()

,Patient_Week,Confidence,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00419637202311204720264_-12,100,ID00419637202311204720264,-12,3020,70.186855,73,Male,Ex-smoker
1,ID00419637202311204720264_-11,100,ID00419637202311204720264,-11,3020,70.186855,73,Male,Ex-smoker
2,ID00419637202311204720264_-10,100,ID00419637202311204720264,-10,3020,70.186855,73,Male,Ex-smoker
3,ID00419637202311204720264_-9,100,ID00419637202311204720264,-9,3020,70.186855,73,Male,Ex-smoker
4,ID00419637202311204720264_-8,100,ID00419637202311204720264,-8,3020,70.186855,73,Male,Ex-smoker


In [9]:
train_df['FROM'] = 'train'
test_df['FROM'] = 'val'
sub_df['FROM'] = 'test'

In [10]:
combined_df = train_df.append([test_df, sub_df])

In [11]:
# initialize base_week column
combined_df['Base_Week'] = combined_df['Weeks']
# make the weeks from sub_df to be np.nan so that when we calculate the base_week it comes from the test_df
combined_df.loc[combined_df['FROM'] == 'test', 'Base_Week'] = np.nan
# now calculate the min for each patient group and set it to the Base_Week column
combined_df['Base_Week'] = combined_df.groupby('Patient')['Base_Week'].transform('min')

In [12]:
# get the base_df (where the Base_Week == the min_week we calculated) so that we can get the base_fvc, base_age and base_percentage
base_df = combined_df[combined_df['Weeks'] == combined_df['Base_Week']]

In [13]:
base_df.rename(columns={
    'FVC': 'Base_FVC',
    'Percent': 'Base_Percent',
    'Age': 'Base_Age'
}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
combined_df = combined_df.merge(base_df[['Patient', 'Base_FVC', 'Base_Percent', 'Base_Age']], on='Patient', how='left')

In [15]:
combined_df['Weeks_Passed'] = combined_df['Weeks'] - combined_df['Base_Week']

In [16]:
MIN_MAX_SCALER.fit(combined_df[combined_df['FROM'] == 'train'][['Weeks_Passed', 'FVC', 'Percent', 'Age']])

MinMaxScaler(copy=True, feature_range=(0, 1))

In [17]:
combined_df[['Weeks_Passed', 'Base_FVC', 'Base_Percent', 'Base_Age']] = MIN_MAX_SCALER.transform(combined_df[['Weeks_Passed', 'Base_FVC', 'Base_Percent', 'Base_Age']])

In [18]:
# convert categoricals into dummies
combined_df['Sex'] = pd.Categorical(combined_df['Sex'], categories=SEX_COLUMNS)
combined_df['SmokingStatus'] = pd.Categorical(combined_df['SmokingStatus'], categories=SMOKING_STATUS_COLUMNS)
combined_df = combined_df.join(pd.get_dummies(combined_df['Sex']))
combined_df = combined_df.join(pd.get_dummies(combined_df['SmokingStatus']))

In [19]:
combined_df.drop_duplicates(inplace=True)

In [20]:
combined_df.reset_index(drop=True)

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,FROM,Patient_Week,Confidence,Base_Week,Base_FVC,Base_Percent,Base_Age,Weeks_Passed,Male,Female,Currently smokes,Ex-smoker,Never smoked
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,train,NaN,NaN,-4.0,0.267050,0.236393,0.769231,0.000000,1,0,0,1,0
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,train,NaN,NaN,-4.0,0.267050,0.236393,0.769231,0.142857,1,0,0,1,0
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,train,NaN,NaN,-4.0,0.267050,0.236393,0.769231,0.174603,1,0,0,1,0
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,train,NaN,NaN,-4.0,0.267050,0.236393,0.769231,0.206349,1,0,0,1,0
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,train,NaN,NaN,-4.0,0.267050,0.236393,0.769231,0.238095,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2265,ID00426637202313170790466,129,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_129,100.0,0.0,0.376525,0.345604,0.615385,2.047619,1,0,0,0,1
2266,ID00426637202313170790466,130,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_130,100.0,0.0,0.376525,0.345604,0.615385,2.063492,1,0,0,0,1
2267,ID00426637202313170790466,131,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_131,100.0,0.0,0.376525,0.345604,0.615385,2.079365,1,0,0,0,1
2268,ID00426637202313170790466,132,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_132,100.0,0.0,0.376525,0.345604,0.615385,2.095238,1,0,0,0,1


In [64]:
TRAIN_PATIENTS = train_df['Patient'].unique().tolist()
# gave the gdcm error
BAD_PATIENT_IDS = ['ID00011637202177653955184', 'ID00052637202186188008618']
ALL_TRAIN_PATIENTS = np.array([pat for pat in TRAIN_PATIENTS if pat not in BAD_PATIENT_IDS])
ALL_TEST_PATIENTS = test_df['Patient'].unique().tolist()

In [22]:
def get_averaged_slices(patient_id, folder_path, num_images):
    # the preprocessed array with NUM_SLICES elements
    # TODO: Handle the case when the NUM_SLICES > the actual total slices
    # TODO: resize the image to 256 X 256?

    full_path = os.path.join(folder_path, patient_id)
    # list of all files in that path and sort them
    all_files = os.listdir(full_path)
    # sorted using the first number part of the file name
    all_files.sort(key = lambda x: int(x.split('.')[0]))

    # read all the dicom files for the patient into the slices list
    slices = [pydicom.read_file(os.path.join(full_path, s)) for s in all_files]
    # sort the slices using their order (file number works too)
    # slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))

    # final array containing averaged num_images images
    out_array = []

    # how many extra files while averaging all images into (num_images) images
    remainder_array_size = len(slices)%num_images

    # how many to average to get a single averaged image
    avging_array_size = len(slices)//num_images

    # get the first one with the remainder images
    first_array = []
    # select the first remainder + avg_arrray_size imgaes and average into one
    for slice in slices[:remainder_array_size+avging_array_size]:
        first_array.append(slice.pixel_array)
    first_avged_array = np.average(first_array, axis=0)
    first_resized = cv2.resize(first_avged_array, (IMG_SIZE, IMG_SIZE))
    out_array.append(first_resized)

    # after the first one get the remaining ones into out_array rolling averaging (avging_array_size) at a time.
    for i in range(remainder_array_size + avging_array_size, len(slices), avging_array_size):
        temp_array = []
        for slice in slices[i:i+avging_array_size]:
            temp_array.append(slice.pixel_array)
        avged_temp_array = np.average(temp_array, axis=0)
        avged_resized = cv2.resize(avged_temp_array, (IMG_SIZE, IMG_SIZE))
        out_array.append(avged_resized)
    
    return np.array(out_array)

In [23]:
array_from_id = {}

In [24]:
# store the train and test images in array_from_id
for id in ALL_TRAIN_PATIENTS:
    array_from_id[id] = get_averaged_slices(id, os.path.join(DATA_DIR, "train"), NUM_IMAGES)

for id in ALL_TEST_PATIENTS:
    array_from_id[id] = get_averaged_slices(id, os.path.join(DATA_DIR, "test"), NUM_IMAGES)

In [43]:
class PulmonaryDataset(Dataset):
    def __init__(self, df, FV, test=False):
        self.df = df
        self.test = test
        self.FV = FV

    def __getitem__(self, idx):
        return {
            'imgarray': torch.from_numpy(array_from_id[self.df.iloc[idx]['Patient']]).unsqueeze(0),
            'features': torch.tensor(self.df[self.FV].iloc[idx].values),
            'target': torch.tensor(self.df['FVC'].iloc[idx])
        }

    def __len__(self):
        return len(self.df)

In [67]:
class PulmonaryModel(nn.Module):
    def __init__(self, cnn_output_size=10, in_features=9, out_quantiles=3):
        super(PulmonaryModel, self).__init__()

        self.conv_layer1 = self._make_conv_layer(1, 8)
        self.conv_layer2 = self._make_conv_layer(8, 32)
        self.conv_layer3 = self._make_conv_layer(32, 64)
        self.conv_layer4 = nn.Conv3d(64, 128, kernel_size=(1, 3, 3))
        self.conv_layer5 = nn.Conv3d(128, 128, kernel_size=(1,3,3), padding=0)

        self.fc1 = nn.Linear(86528, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, cnn_output_size)
        
        self.fc4 = nn.Linear(cnn_output_size + in_features, 100)
        self.fc5 = nn.Linear(100, out_quantiles)

    def _make_conv_layer(self, in_c, out_c):
        return nn.Sequential(
            nn.Conv3d(in_c, out_c, kernel_size=(2,3,3), padding=0),
            nn.LeakyReLU(),
            nn.Conv3d(out_c, out_c, kernel_size=(2, 3, 3), padding=1),
            nn.LeakyReLU(),
            nn.MaxPool3d((2,2,2)),
        )

    def forward(self, x, y):
        x = self.conv_layer1(x)
        x = self.conv_layer2(x)
        x = self.conv_layer3(x)
        x = self.conv_layer4(x)
        x = self.conv_layer5(x)

        # flatten
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)

        # concatenate the in_features
        x = torch.cat((x, y), dim=1)
        x = self.fc4(x)
        x = F.relu(x)
        x = self.fc5(x)
        return x

In [27]:
clip_one, clip_two = torch.tensor(70, dtype=torch.float32).to(DEVICE), torch.tensor(1000, dtype=torch.float32).to(DEVICE)
def score(y_true, y_pred):
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]

    sigma_clip = torch.max(sigma, clip_one)
    delta = torch.abs(y_true - fvc_pred)
    delta = torch.min(delta, clip_two)
    sq2 = torch.sqrt(torch.tensor(2, dtype=torch.float32))
    metric = (delta / sigma_clip) * sq2 + torch.log(sigma_clip * sq2)
    return torch.mean(metric)

In [28]:
def quantile_loss(preds, target, quantiles, _lambda):
    assert not target.requires_grad
    assert preds.size(0) == target.size(0)
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i]
        losses.append(torch.max((q - 1) * errors, q * errors).unsqueeze(1))
    loss = torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))
    return loss
    # return _lambda * loss + (1 - _lambda) * score(target, preds)

In [29]:
class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [35]:
def train_one_epoch(model, train_data_loader, optimizer, train_loss):
    model.train()
    for i, data in enumerate(train_data_loader):
        images = data['imgarray']
        features = data['features']
        targets = data['target']

        images = images.to(DEVICE).float()
        features = features.to(DEVICE).float()
        targets = targets.to(DEVICE).float()

        model.zero_grad()
        out = model(images, features)
        loss = quantile_loss(out, targets, QUANTILES, 0.6)
        train_loss.update(loss, features.size(0))
        loss.backward()
        optimizer.step()

In [36]:
def eval_one_epoch(model, valid_data_loader, valid_loss, lr_scheduler):
    model.eval()

    with torch.no_grad():
        for i, data in enumerate(valid_data_loader):
            images = data['imgarray']
            features = data['features']
            targets = data['target']

            images = images.to(DEVICE).float()
            features = features.to(DEVICE).float()
            targets = targets.to(DEVICE).float()
            
            out = model(images, features)
            loss = quantile_loss(out, targets, QUANTILES, 0.6)
            valid_loss.update(loss, features.size(0))
    
    if lr_scheduler is not None:
        lr_scheduler.step(valid_loss.avg)

In [66]:
for fold, (train_index, test_index) in enumerate(kf.split(ALL_TRAIN_PATIENTS)):
    model = PulmonaryModel(10, len(FV))
    model = model.to(DEVICE)

    train_ids = ALL_TRAIN_PATIENTS[train_index]
    test_ids = ALL_TRAIN_PATIENTS[test_index]

    df_train = combined_df[combined_df['Patient'].isin(train_ids)].reset_index(drop=True)
    df_valid = combined_df[combined_df['Patient'].isin(test_ids)].reset_index(drop=True)

    train_dataset = PulmonaryDataset(df_train, FV)
    valid_dataset = PulmonaryDataset(df_valid, FV)

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=10,
        shuffle=True,
        num_workers=4
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=4,
        shuffle=False,
        num_workers=4
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=50, factor=0.7, verbose=True)

    best_valid_loss = float('inf')

    
    # tq = tqdm(range(NUM_EPOCHS), desc=f"Fold {fold}")
    for epoch in range(NUM_EPOCHS):
        train_loss = AverageMeter()
        valid_loss = AverageMeter()

        train_one_epoch(model, train_data_loader, optimizer, train_loss)
        eval_one_epoch(model, valid_data_loader, valid_loss, lr_scheduler)

        if epoch % PRINT_EVERY == 0:
            print(f"Epoch {epoch}/{NUM_EPOCHS}, Loss {train_loss.avg}")
            print(f"Fold {fold}, Valid Loss {valid_loss.avg} \n")
        
        # tq.set_postfix(val_loss=valid_loss.avg.item())

        if valid_loss.avg < best_valid_loss:
            best_valid_loss = valid_loss.avg
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, os.path.join(CONFIG.CFG.DATA.MODELS_OUT, f"model_fold_{fold}.pt"))

torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 10])
torch.Size([10, 9])
torch.Size([10, 

KeyboardInterrupt: 

In [ ]:
train_dataset = PulmonaryDataset(train_df, FV, test=False)

train_data_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4,
)

In [ ]:
for i, data in enumerate(train_data_loader):
    imgarray = data['imgarray'].to(DEVICE).float()
    out = model(imgarray)
    print(out)
    break

In [ ]:
for key in array_from_id:
    print(array_from_id[key].shape)